In [4]:
!pip install deep-translator


In [2]:
!pip install scikit-learn


In [7]:
import pandas as pd

df_origem = pd.read_csv("/content/drive/MyDrive/data_bases/doencas_sintomas.csv")  # Substitua pelo nome do seu arquivo
df_origem.head()


,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive


In [8]:
df = df_origem.copy()

In [9]:
from deep_translator import GoogleTranslator

def traduzir_deep(texto):
    try:
        return GoogleTranslator(source='en', target='pt').translate(texto)
    except Exception as e:
        print(f"Erro ao traduzir {texto}: {e}")
        return texto


In [10]:
# Traduz colunas
colunas_traduzidas = {col: traduzir_deep(col) for col in df.columns}
df.rename(columns=colunas_traduzidas, inplace=True)

# Traduz valores da coluna 'Doença'
df['Doença'] = df['Doença'].apply(traduzir_deep)


In [11]:
# NÃO EXECUTAR ESSA LINHA DEPOIS DE 1 VEZ!
df_traduzido = df.copy()


In [13]:
df.to_csv('/content/drive/MyDrive/data_bases/doencas_sintomas_traduzido.csv', index=False)

In [16]:
# DEPOIS DE INSTALAR AS BIBLIOTECAS, COMERÇAR POR AQUI!!!
df = pd.read_csv("/content/drive/MyDrive/data_bases/doencas_sintomas_traduzido.csv")

In [18]:
df = df[df['Variável de resultado'] == 'Positive']

In [19]:
df.head()

,Doença,Febre,Tosse,Fadiga,Dificuldade em respirar,Idade,Gênero,Pressão arterial,Nível de colesterol,Variável de resultado
0,Gripe,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
3,Asma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Asma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
5,Eczema,Yes,No,No,No,25,Female,Normal,Normal,Positive
6,Gripe,Yes,Yes,Yes,Yes,25,Female,Normal,Normal,Positive


In [20]:
print("Existem valores ausentes? ", df.isnull().values.any())
print("\nQuantidade de valores ausentes por coluna:")
print(df.isnull().sum())
print("\nPercentual de valores ausentes por coluna:")
print((df.isnull().mean() * 100).round(2))


Existem valores ausentes?  False

Quantidade de valores ausentes por coluna:
Doença                     0
Febre                      0
Tosse                      0
Fadiga                     0
Dificuldade em respirar    0
Idade                      0
Gênero                     0
Pressão arterial           0
Nível de colesterol        0
Variável de resultado      0
dtype: int64

Percentual de valores ausentes por coluna:
Doença                     0.0
Febre                      0.0
Tosse                      0.0
Fadiga                     0.0
Dificuldade em respirar    0.0
Idade                      0.0
Gênero                     0.0
Pressão arterial           0.0
Nível de colesterol        0.0
Variável de resultado      0.0
dtype: float64


In [21]:
# Função para limpeza segura
def limpar_binario(valor):
    if isinstance(valor, str):
        valor = valor.strip().capitalize()
        return {'Yes': 1, 'No': 0}.get(valor, None)
    return None

In [22]:
# Binário: Sim/Não
df['Febre'] = df['Febre'].apply(limpar_binario)
df['Tosse'] = df['Tosse'].apply(limpar_binario)
df['Fadiga'] = df['Fadiga'].apply(limpar_binario)
df['Dificuldade em respirar'] = df['Dificuldade em respirar'].apply(limpar_binario)

# Gênero
df['Gênero'] = df['Gênero'].map({'Male': 1, 'Female': 0})

# Pressão Arterial e Colesterol (ordinal)
map_ordinal = {'Low': 0, 'Normal': 1, 'High': 2}
df['Pressão arterial'] = df['Pressão arterial'].map(map_ordinal)
df['Nível de colesterol'] = df['Nível de colesterol'].map(map_ordinal)

# Variável de resultado (alvo)
df['Diagnóstico'] = df['Variável de resultado'].map({'Positive': 1, 'Negative': 0})

df.head()


,Doença,Febre,Tosse,Fadiga,Dificuldade em respirar,Idade,Gênero,Pressão arterial,Nível de colesterol,Variável de resultado,Diagnóstico
0,Gripe,1,0,1,1,19,0,0,1,Positive,1
3,Asma,1,1,0,1,25,1,1,1,Positive,1
4,Asma,1,1,0,1,25,1,1,1,Positive,1
5,Eczema,1,0,0,0,25,0,1,1,Positive,1
6,Gripe,1,1,1,1,25,0,1,1,Positive,1


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Features e alvo
X = df[['Febre', 'Tosse', 'Fadiga', 'Dificuldade em respirar', 'Idade',
        'Gênero', 'Pressão arterial', 'Nível de colesterol']]
y = df['Doença']

# Split 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelos
modelos = {
    "Decision Tree": DecisionTreeClassifier(max_depth=4, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
#    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
}

# Treinar e avaliar todos
for nome, modelo in modelos.items():
    modelo.fit(X, y)
    y_pred = modelo.predict(X_test)

    print(f"\n===== {nome} =====")
    print(f"Acurácia: {accuracy_score(y_test, y_pred):.2%}")
    print("Relatório de classificação:")
    print(classification_report(y_test, y_pred))



===== Decision Tree =====
Acurácia: 13.16%
Relatório de classificação:
                                           precision    recall  f1-score   support

                                      AVC       0.17      0.50      0.25         2
                                     Asma       0.11      1.00      0.20         1
                                Bronquite       1.00      0.50      0.67         2
                                  Caxumba       0.00      0.00      0.00         1
                        Colite ulcerativa       0.00      0.00      0.00         1
                        Câncer colorretal       0.00      0.00      0.00         1
                       Câncer de pâncreas       0.00      0.00      0.00         1
                            Câncer de rim       0.00      0.00      0.00         1
                        Câncer testicular       0.00      0.00      0.00         1
                                 Diabetes       0.00      0.00      0.00         1
    Distúrbios

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [25]:
import numpy as np

# Prever probabilidades
for nome, modelo in modelos.items():
    modelo.fit(X, y)
    y_pred = modelo.predict_proba(X_test)

    probabilidades = modelo.predict_proba(X_test)

    # Nome das classes (doenças)
    classes = modelo.classes_

    # Para cada paciente no teste, pega Top 3 doenças com maiores chances
    top_n = 3
    top_doencas = np.argsort(probabilidades, axis=1)[:, -top_n:][:, ::-1]  # ordenado decrescente

    # Traduz índice para nome da doença
    top_doencas_nomes = [[classes[i] for i in linha] for linha in top_doencas]

print(classes)
print(top_n)
print(top_doencas)
print(top_doencas_nomes)
print(probabilidades)

['AVC' 'Amidalite' 'Anemia das células falciformes' 'Artrite reumatoide'
 'Asma' 'Bronquite' 'Catapora' 'Cataratas' 'Caxumba' 'Cirrose'
 'Colite ulcerativa' 'Conjuntivite (olho rosa)' 'Câncer colorretal'
 'Câncer de fígado' 'Câncer de pulmão' 'Câncer de pâncreas'
 'Câncer de rim' 'Câncer de tireóide' 'Câncer testicular' 'Demência'
 'Depressão' 'Diabetes' 'Distúrbios alimentares (anorexia, ...'
 'Diverticulite' 'Doença arterial coronária'
 'Doença cardíaca hipertensiva' 'Doença de Alzheimer' 'Doença de Crohn'
 'Doença de Lyme' 'Doença de Parkinson' 'Doença hepática'
 'Doença pulmonar obstrutiva crônica (DPOC)' 'Doença renal'
 'Doença renal crônica' 'Eczema' 'Enxaqueca' 'Esclerose múltipla'
 'Escoliose' 'Espinha bífida' 'Febre tifóide' 'Fibromialgia'
 'Gastroenterite' 'Gota' 'Gripe' 'Gripe comum' 'Hemofilia' 'Hemorróidas'
 'Hepatite B.' 'Hipertensão' 'Hipertireoidismo' 'Hipoglicemia'
 'Hipotireoidismo' 'Infarto do miocárdio (coração ...'
 'Infecção do trato urinário' 'Infecção do trato u

In [31]:
import numpy as np

# Prever probabilidades
for nome, modelo in modelos.items():
    modelo.fit(X, y)
    probabilidades = modelo.predict_proba(X_test)
    classes = modelo.classes_

    top_n = 3
    top_doencas_idx = np.argsort(probabilidades, axis=1)[:, -top_n:][:, ::-1]

    print(f"\nModelo: {nome}")
    for i, indices in enumerate(top_doencas_idx):
        print(f"\nPaciente {i+1} (entrada {X_test.iloc[i].to_dict()}):")
        for j in indices:
            print(f"  {classes[j]}: {probabilidades[i][j]:.2%}")


<class 'pandas.core.frame.DataFrame'>

Modelo: Decision Tree

Paciente 1 (entrada {'Febre': 0, 'Tosse': 1, 'Fadiga': 0, 'Dificuldade em respirar': 0, 'Idade': 50, 'Gênero': 0, 'Pressão arterial': 2, 'Nível de colesterol': 2}):
  Osteoporose: 5.10%
  Hipertensão: 5.10%
  Hipotireoidismo: 4.08%

Paciente 2 (entrada {'Febre': 1, 'Tosse': 1, 'Fadiga': 1, 'Dificuldade em respirar': 0, 'Idade': 65, 'Gênero': 0, 'Pressão arterial': 1, 'Nível de colesterol': 2}):
  AVC: 15.00%
  Doença de Alzheimer: 15.00%
  Caxumba: 10.00%

Paciente 3 (entrada {'Febre': 0, 'Tosse': 0, 'Fadiga': 1, 'Dificuldade em respirar': 0, 'Idade': 65, 'Gênero': 1, 'Pressão arterial': 1, 'Nível de colesterol': 2}):
  AVC: 15.00%
  Doença de Alzheimer: 15.00%
  Caxumba: 10.00%

Paciente 4 (entrada {'Febre': 1, 'Tosse': 1, 'Fadiga': 1, 'Dificuldade em respirar': 0, 'Idade': 45, 'Gênero': 0, 'Pressão arterial': 1, 'Nível de colesterol': 2}):
  Osteoporose: 5.10%
  Hipertensão: 5.10%
  Hipotireoidismo: 4.08%

Paciente 5 (entr

In [30]:
import joblib
print(modelos['Random Forest'])
# Salvar o modelo treinado
joblib.dump(modelos['Random Forest'], '/content/drive/MyDrive/data_bases/modelo_triagem.pkl')

RandomForestClassifier(random_state=42)


['/content/drive/MyDrive/data_bases/modelo_triagem.pkl']